In [103]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [102]:
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

### Extract PDFs

In [86]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    document = loader.load()
    
    return document

In [87]:
extracted_data = load_pdf('../data')

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 10 65536 (offset 0)
Ignoring wrong pointing object 21 65536 (offset 0)
Ignoring wrong pointing object 25 65536 (offset 0)
Ignoring wrong pointing object 40 65536 (offset 0)
Ignoring wrong pointing object 57 65536 (offset 0)
Ignoring wrong pointing object 73 65536 (offset 0)
Ignoring wrong pointing object 81 65536 (offset 0)
Ignoring wrong pointing object 92 65536 (offset 0)
Ignoring wrong pointing object 103 65536 (offset 0)
Ignoring wrong pointing object 108 65536 (offset 0)
Ignoring wrong pointing object 113 65536 (offset 0)
Ignoring wrong pointing object 118 65536 (offset 0)
Ignoring wrong pointing object 123 65536 (offset 0)
Ignoring wrong pointing object 128 65536 (offset 0)
Ignoring wrong pointing object 133 65536 (offset 0)
Ignoring wrong pointing object 138 65536 (offset 0)
Ignoring wrong pointing object 143 65536 (offset 0)
Ignoring wrong pointing object 148 65536 (offset 0)
Ignoring wrong pointin

### Creating text chunks

In [88]:
def text_split(extracted_data):
    splited_text = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
    text_chunk = splited_text.split_documents(extracted_data)
    
    return text_chunk

In [89]:
text_chunk = text_split(extracted_data)
len(text_chunk)

5862

### Download HuggingFace Embedding

In [90]:
def download_hf_embeddings():
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding

In [91]:
embedding = download_hf_embeddings()

In [92]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Pinecone Database

In [93]:
# Create the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Access the index
index = pc.Index("qchatbot")


In [94]:
texts = [t.page_content for t in text_chunk]

# Create or connect to the vector store
doc_search = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embedding,
    index_name="qchatbot",
    namespace="default"  # optional
)

### Similarity search

In [96]:
query = "What is reversible gate?"
docs = doc_search.similarity_search(query=query, k=3)
print(docs)

[Document(id='fc161a24-c7ac-4aae-88d9-f6f7a4f5285f', metadata={}, page_content='A reversible gate, is a logic gate where, given the output(s) of the gate, we can\nalways determine what the input(s) was (were). An example is the NOT gate:'), Document(id='910b6f05-c254-4b3f-85e3-c3a99af1e133', metadata={}, page_content='1.5 Reversible Logic Gates 49\nWe can generalize this technique several ways. First, the gate could be a function\nof any number of variables. For example, say we have a gate with three inputsA, B,\nC, and one output f (A,B,C):\nGate\nA\nB\nC\nf(A, B, C)\nThis must be irreversible because there are fewer outputs than inputs. To make it\nreversible, we add a fourth input D that we XOR with f (A,B,C):\nA A\nB B'), Document(id='a1d6dd9b-7952-4e71-a575-db709b6dd67f', metadata={}, page_content='Another way to contrast reversible and irreversible gates is whether information\nis lost. With a reversible gate, no information is lost since we can always recover\nthe inputs from th

In [97]:
template = """
You are a knowledgeable quantum assistant focused strictly on topics related to **quantum mechanics** and **quantum computing**.

Use the provided context to answer the user's question. If the context does not contain relevant information to answer the question, respond with:
**"I'm not sure about that based on the information I have."**

Strictly avoid making up information or answering unrelated topics.

Context:
{context}

Question:
{question}

Answer:
""".strip()

In [98]:
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
    )

chain_type_kwargs = {'prompt': PROMPT}

In [104]:
llm = ChatOpenAI(
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="shisa-ai/shisa-v2-llama3.3-70b:free",
    temperature=0.5,
)

In [105]:
qna = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type='stuff',
                                  retriever=doc_search.as_retriever(search_kwargs={'k':2}),
                                  return_source_documents=True,
                                  chain_type_kwargs=chain_type_kwargs
                                  )

In [ ]:
while True:
    user_input = input(f"Input Prompt: ")
    result=qna({'query':user_input})
    
    print(result['result'])

C:\Users\Durjay\AppData\Local\Temp\ipykernel_4716\3434308939.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qna({'query':user_input})


A reversible gate is a logic gate where, given the output(s) of the gate, we can always determine what the input(s) was (were). This means that the gate’s operation can be inverted to recover the original inputs from the outputs, ensuring no information is lost during the computation.  

For example, the NOT gate is a simple reversible gate because knowing the output allows us to uniquely determine the input.  

More generally, a reversible gate must have at least as many outputs as inputs to avoid information loss. If a gate has fewer outputs than inputs (like the example with inputs *A, B, C* and one output *f(A, B, C)*), it is irreversible because multiple input combinations could produce the same output.  

To make such a gate reversible, additional inputs (or "ancillary bits") can be introduced, such as in the modified example where a fourth input *D* is XORed with *f(A, B, C)* to ensure reversibility.
Here are five examples of reversible logic gates based on the given context:  
